In [29]:
import pandas as pd
import streamlit as st
import plotly.express as px
import altair as alt


In [30]:
df = pd.read_csv("vehicles_us.csv") 
df.head() 

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [31]:
df = df.drop_duplicates().reset_index(drop=True)

In [32]:
#standardise all columns and stripping 
for col in ["model", "condition", "fuel", "transmission", "type", "paint_color"]:
    df[col] = df[col].str.lower().str.strip()

In [33]:
#recasting data types
df["date_posted"] = pd.to_datetime(df["date_posted"], errors="coerce")
df["is_4wd"]      = df["is_4wd"].fillna(0).astype(bool)   # NaN→False

In [34]:
#numeric columns built for staying integers
int_cols   = {"model_year": "model", "cylinders": "fuel"}     # {col: group_key}
for col, grp in int_cols.items():
    # temp float to accept non‑whole medians
    df[col] = df[col].astype(float)
    df[col] = (
        df.groupby(grp)[col]
          .transform(lambda s: s.fillna(round(s.median())))
    )
    df[col] = df[col].astype("Int64")  # back to nullable int

In [35]:
#added features 
CURRENT_YEAR = 2025
df["vehicle_age"] = CURRENT_YEAR - df["model_year"]
df["days_since_post"] = (pd.Timestamp("today") - df["date_posted"]).dt.days

In [36]:
# Categorical NaN → "unknown"
df["paint_color"] = df["paint_color"].fillna("unknown")

In [37]:
#sanity caps 
df["price"] = df["price"].clip(lower=500, upper=120_000)

In [38]:
#auditing
display(df.info())
display(df.describe(include="all").T.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   price            51525 non-null  int64         
 1   model_year       51525 non-null  Int64         
 2   model            51525 non-null  object        
 3   condition        51525 non-null  object        
 4   cylinders        51525 non-null  Int64         
 5   fuel             51525 non-null  object        
 6   odometer         43633 non-null  float64       
 7   transmission     51525 non-null  object        
 8   type             51525 non-null  object        
 9   paint_color      51525 non-null  object        
 10  is_4wd           51525 non-null  bool          
 11  date_posted      51525 non-null  datetime64[ns]
 12  days_listed      51525 non-null  int64         
 13  vehicle_age      51525 non-null  Int64         
 14  days_since_post  51525 non-null  int64

None

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
price,51525.0,NaN,NaN,NaN,12123.021679,500.0,5000.0,9000.0,16839.0,120000.0,9698.364543
model_year,51525.0,<NA>,<NA>,<NA>,2009.793557,1908.0,2007.0,2011.0,2014.0,2019.0,6.099381
model,51525,100,ford f-150,2796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,51525,6,excellent,24773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,51525.0,<NA>,<NA>,<NA>,6.12623,3.0,4.0,6.0,8.0,12.0,1.583377
fuel,51525,5,gas,47288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
odometer,43633.0,NaN,NaN,NaN,115553.461738,0.0,70000.0,113000.0,155000.0,990000.0,65094.611341
transmission,51525,3,automatic,46902,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,51525,13,suv,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN
paint_color,51525,13,white,10029,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#missed odometer cleaning here 
age_bins = pd.cut(CURRENT_YEAR - df["model_year"].astype(float),
                  [0,5,10,15,20,100])

df["odometer"] = (
    df.groupby(age_bins)["odometer"]
      .transform(lambda s: s.fillna(s.median()))
)
df["odometer"] = df["odometer"].clip(upper=500_000)
observed = True

C:\Users\falna\AppData\Local\Temp\ipykernel_24048\133263461.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(age_bins)["odometer"]


In [40]:
#auditing again 
display(df.info())
display(df.describe(include="all").T.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   price            51525 non-null  int64         
 1   model_year       51525 non-null  Int64         
 2   model            51525 non-null  object        
 3   condition        51525 non-null  object        
 4   cylinders        51525 non-null  Int64         
 5   fuel             51525 non-null  object        
 6   odometer         51523 non-null  float64       
 7   transmission     51525 non-null  object        
 8   type             51525 non-null  object        
 9   paint_color      51525 non-null  object        
 10  is_4wd           51525 non-null  bool          
 11  date_posted      51525 non-null  datetime64[ns]
 12  days_listed      51525 non-null  int64         
 13  vehicle_age      51525 non-null  Int64         
 14  days_since_post  51525 non-null  int64

None

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
price,51525.0,NaN,NaN,NaN,12123.021679,500.0,5000.0,9000.0,16839.0,120000.0,9698.364543
model_year,51525.0,<NA>,<NA>,<NA>,2009.793557,1908.0,2007.0,2011.0,2014.0,2019.0,6.099381
model,51525,100,ford f-150,2796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,51525,6,excellent,24773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,51525.0,<NA>,<NA>,<NA>,6.12623,3.0,4.0,6.0,8.0,12.0,1.583377
fuel,51525,5,gas,47288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
odometer,51523.0,NaN,NaN,NaN,114986.378666,0.0,71000.0,110439.0,151890.0,500000.0,60879.967562
transmission,51525,3,automatic,46902,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,51525,13,suv,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN
paint_color,51525,13,white,10029,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
missing_rows = df[df["odometer"].isna()]
display(missing_rows[["price", "model", "fuel", "model_year",
                      "vehicle_age", "odometer"]].head())
print(f"Rows with NaN odometer: {len(missing_rows)}")

,price,model,fuel,model_year,vehicle_age,odometer
33906,12995,gmc yukon,gas,1908,117,NaN
33907,12995,cadillac escalade,gas,1908,117,NaN


Rows with NaN odometer: 2


In [44]:
df["odometer"] = df["odometer"].fillna(df["odometer"].median())

In [45]:
print(df["odometer"].isna().sum()) 

0


In [46]:
#auditing again 
display(df.info())
display(df.describe(include="all").T.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   price            51525 non-null  int64         
 1   model_year       51525 non-null  Int64         
 2   model            51525 non-null  object        
 3   condition        51525 non-null  object        
 4   cylinders        51525 non-null  Int64         
 5   fuel             51525 non-null  object        
 6   odometer         51525 non-null  float64       
 7   transmission     51525 non-null  object        
 8   type             51525 non-null  object        
 9   paint_color      51525 non-null  object        
 10  is_4wd           51525 non-null  bool          
 11  date_posted      51525 non-null  datetime64[ns]
 12  days_listed      51525 non-null  int64         
 13  vehicle_age      51525 non-null  Int64         
 14  days_since_post  51525 non-null  int64

None

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
price,51525.0,NaN,NaN,NaN,12123.021679,500.0,5000.0,9000.0,16839.0,120000.0,9698.364543
model_year,51525.0,<NA>,<NA>,<NA>,2009.793557,1908.0,2007.0,2011.0,2014.0,2019.0,6.099381
model,51525,100,ford f-150,2796,NaN,NaN,NaN,NaN,NaN,NaN,NaN
condition,51525,6,excellent,24773,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cylinders,51525.0,<NA>,<NA>,<NA>,6.12623,3.0,4.0,6.0,8.0,12.0,1.583377
fuel,51525,5,gas,47288,NaN,NaN,NaN,NaN,NaN,NaN,NaN
odometer,51525.0,NaN,NaN,NaN,114986.202154,0.0,71000.0,110439.0,151890.0,500000.0,60878.792558
transmission,51525,3,automatic,46902,NaN,NaN,NaN,NaN,NaN,NaN,NaN
type,51525,13,suv,12405,NaN,NaN,NaN,NaN,NaN,NaN,NaN
paint_color,51525,13,white,10029,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:


fig = px.histogram(
    df, x="price",
    nbins=60,
    facet_col="condition",
    title="Price Distribution by Vehicle Condition",
    labels={"price": "Price (USD)"}
)
fig.show()

In [53]:
CURRENT_YEAR = 2025          # keep this in one place

# --- 1. Filter df for valid years 1980‑CURRENT_YEAR -------------
mask = (
    df["model_year"].notna() &
    (df["model_year"] >= 1980) &
    (df["model_year"] <= CURRENT_YEAR)
)

year_counts = (
    df.loc[mask]                   # apply the filter first
      .groupby("model_year")
      .size()
      .reset_index(name="count")
      .sort_values("model_year")
)

# --- 2. Plot -----------------------------------------------------
fig = px.scatter(                 # or px.bar(...) if you prefer bars
    year_counts,
    x="model_year",
    y="count",
    size="count",
    title="Number of Listings by Model Year (1980–2025)",
    labels={"model_year": "Model Year", "count": "Number of Cars Listed"},
)
fig.update_traces(marker=dict(opacity=0.7))
fig.show()